In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm import load_node
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.calculation.work import WorkCalculation
from aiida.orm.calculation.job import JobCalculation

import numpy as np
import scipy.constants as const
import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import re
import gzip
import matplotlib.pyplot as plt
from collections import OrderedDict
import urlparse
import io

import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def get_calc_by_label(workcalc, label):
    qb = QueryBuilder()
    qb.append(WorkCalculation, filters={'uuid':workcalc.uuid})
    qb.append(JobCalculation, output_of=WorkCalculation, filters={'label':label})
    assert qb.count() == 1
    calc = qb.first()[0]
    assert(calc.get_state() == 'FINISHED')
    return calc

In [ ]:
def read_and_process_pdos_file(pdos_path):
    header = open(pdos_path).readline()
    fermi = float(re.search("Fermi.* ([+-]?[0-9]*[.]?[0-9]+)", header).group(1))
    try:
        kind = re.search("atomic kind.(\S+)", header).group(1)
    except:
        kind = None
    data = np.loadtxt(pdos_path)
    out_data = np.zeros((data.shape[0], 2))
    out_data[:, 0] = (data[:, 1] - fermi) * 27.21138602 # energy
    out_data[:, 1] = np.sum(data[:, 3:], axis=1) # "contracted pdos"
    return out_data, kind
    

def process_pdos_files(scf_calc):
    retr_files = scf_calc.out.retrieved.get_folder_list()
    abs_path = scf_calc.out.retrieved.get_abs_path() + "/path/"
    mol_pdos = None
    kind_pdos = {}
    for retr_file in retr_files:
        if retr_file.startswith('aiida-list'):
            mol_pdos, _ = read_and_process_pdos_file(abs_path + retr_file)
        if retr_file.startswith('aiida-k'):
            k_pdos, kind = read_and_process_pdos_file(abs_path + retr_file)
            kind_pdos[kind] = k_pdos
    return mol_pdos, kind_pdos

In [ ]:
mol_pdos = None
kind_pdos = None
tdos = None
ovl_matrix = None
ovl_energies = None
ovl_gas_homo = None
ovl_gas_energies = None

def load_pk(b):
    global mol_pdos, kind_pdos, tdos
    global ovl_matrix, ovl_energies, ovl_gas_homo, ovl_gas_energies
    try:
        workcalc = load_node(pk=pk_select.value)
        slab_scf_calc = get_calc_by_label(workcalc, 'slab_scf')
        overlap_calc = get_calc_by_label(workcalc, 'overlap')
    except:
        print("Incorrect pk.")
        return
    
    # load pdos
    mol_pdos, kind_pdos = process_pdos_files(slab_scf_calc)
    tdos = np.zeros(mol_pdos.shape)
    tdos[:, 0] = mol_pdos[:, 0]
    for kp in kind_pdos.items():
        tdos[:, 1] += kp[1][:, 1]
    
    # load overlap
    overlap_data = np.load(overlap_calc.out.retrieved.get_abs_path('overlap.npz'))
    ovl_matrix = overlap_data['overlap_matrix']
    ovl_energies = overlap_data['en_grp2']
    ovl_gas_homo = int(overlap_data['homo_grp1'])
    ovl_gas_energies = overlap_data['en_grp1']
    
    initialize_selections()

pk_select = ipw.IntText(value=0, description='pk')

load_pk_btn = ipw.Button(description='Load pk')
load_pk_btn.on_click(load_pk)
display(pk_select, load_pk_btn)

# PDOS and overlap

In [ ]:
def create_series_w_broadening(x_values, y_values, x_arr, fwhm, shape='g'):
    spectrum = np.zeros(len(x_arr))
    def lorentzian(x_):
        factor = np.pi*fwhm/2 # to make maximum 1.0
        return factor*0.5*fwhm/(np.pi*(x_**2+(0.5*fwhm)**2))
    def gaussian(x_):
        sigma = fwhm/2.3548
        return np.exp(-x_**2/(2*sigma**2))
    for xv, yv in zip(x_values, y_values):
        if shape == 'g':
            spectrum += yv*gaussian(x_arr - xv)
        else:
            spectrum += yv*lorentzian(x_arr - xv)
    return spectrum

In [ ]:
def create_the_plot():
    
    fwhm = fwhm_slider.value
    de = np.min([fwhm/5, 0.005])
    elim = energy_range_slider.value
    energy_arr = np.arange(elim[0], elim[1], de)
    
    plt.figure(figsize=(12, 6))
    
    ### -----------------------------------------------
    ### Pdos part
    tdos_series = create_series_w_broadening(tdos[:, 0], tdos[:, 1], energy_arr, fwhm)
    mol_pdos_series = create_series_w_broadening(mol_pdos[:, 0], mol_pdos[:, 1], energy_arr, fwhm)
    
    tdos_factor = 0.1
    
    ax1 = plt.gca()

    ax1.plot(energy_arr, tdos_factor*tdos_series, 'gray', label=r'$%.1f\cdot$ TDOS'%tdos_factor)
    ax1.fill_between(energy_arr, 0.0, tdos_factor*tdos_series, facecolor='lightgray')

    ax1.plot(energy_arr, mol_pdos_series, 'k', label='mol')
    ax1.fill_between(energy_arr, 0.0, mol_pdos_series, facecolor='k', alpha=0.3)

    ax1.set_xlim(elim)
    ax1.set_ylim([0.0, np.max(tdos_factor*tdos_series)])
    ax1.set_ylabel("DOS [a.u.]")

    ax1.legend(loc='upper left')
    
    ### -----------------------------------------------
    ### overlap part
    
    ax2 = ax1.twinx()
    
    max_overlap = 0.0

    for i_gas in range(ovl_matrix.shape[0]):
        wrt_h = i_gas - ovl_gas_homo
        if wrt_h < 0:
            label = "HOMO%d"%wrt_h
        elif wrt_h == 0:
            label = "HOMO"
        elif wrt_h == 1:
            label = "LUMO"
        else:
            label = "LUMO+%d"%(wrt_h-1)
        label += " (%.2f)"%ovl_gas_energies[i_gas]
        series = create_series_w_broadening(ovl_energies, ovl_matrix[i_gas], energy_arr, fwhm)
        if np.max(series) > max_overlap:
            max_overlap = np.max(series)
        ax2.plot(energy_arr, series, label=label, lw=2.0)
    
    overlap_lim = np.min([np.around(max_overlap+0.055, 1), 1.0])
    ax2.set_ylim([0.0, overlap_lim])
    ax2.set_ylabel("Overlap of orbitals")
    ax2.legend(loc='upper right')

    ax1.set_xlabel("$E-E_F$ [eV]")

    plt.show()

In [ ]:
def initialize_selections():
    min_e = np.around(np.min(ovl_energies), 1)
    max_e = np.around(np.max(ovl_energies), 1)
    
    energy_range_slider.min = min_e
    energy_range_slider.max = max_e
    energy_range_slider.value = [min_e, max_e]
    
def make_plot(b):
    with plot_output:
        create_the_plot()
def clear_plot(b):
    with plot_output:
        clear_output()

style = {'description_width': '140px'}
layout = {'width': '50%'}

fwhm_slider = ipw.FloatSlider(
    value=0.05,
    min=0.01,
    max=0.2,
    step=0.01,
    description='broadening fwhm (eV)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style=style,
    layout=layout
)

energy_range_slider = ipw.FloatRangeSlider(
    value=[0.0, 0.0],
    min=0.0,
    max=0.0,
    step=0.1,
    description='energy range (eV)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style=style,
    layout=layout
)

plot_output = ipw.Output()
plot_btn = ipw.Button(description="plot")
plot_btn.on_click(make_plot)
clear_btn = ipw.Button(description="clear")
clear_btn.on_click(clear_plot)

display(fwhm_slider, energy_range_slider, ipw.HBox([plot_btn, clear_btn]), plot_output)

In [ ]:
### Load the URL after everything is set up ###
try:
    url = urlparse.urlsplit(jupyter_notebook_url)
    pk_select.value = urlparse.parse_qs(url.query)['pk'][0]
    load_pk(0)
except:
    pass